In [8]:
import pygame
import random

In [9]:
### Create Q-Learning Table

actions = [0,1]

buckets = 10
Q_table = {}
for i in range(-600, 1, buckets): 
    for x in range(-50, 440, buckets): 
        for j in actions:
            Q_table[((i,x),j)] = 0


def get_bucket(state):
    diffy, diffx = state
    if diffy < -600:
        diffy = -600
    elif diffy > 0:
        diffy = 0
    
    if diffx < -50:
        diffx = -50
    elif diffx > 400:
        diffx = 400

    return buckets*int(diffy/buckets), buckets*int(diffx/buckets)

exploration_rate = 0.000
discount_factor = 0



# Choose the action according an espilon-greedy policy
def Action_RL(state):
    if random.random() < exploration_rate:
        return random.choice(actions)
    else:
        return max(actions, key = lambda a: Q_table[get_bucket(state), a])

# Update Q-table with bellman equation
def Update_RL(state, action, reward, next_state):
    Q_table[get_bucket(state), action] += 0.1*(reward + discount_factor*max(Q_table[get_bucket(next_state), a] for a in actions) - Q_table[get_bucket(state), action])




    

In [10]:
#save Q table 



import pickle
#load Q table
with open('Q_table_trained.pkl', 'rb') as f:
    Q_table = pickle.load(f)


exploration_rate = 0.0
discount_factor = 0.0

def Update_RL(state, action, reward, next_state):
    pass



In [12]:
def launch_game(fps, verbose=False):

    FPS = fps
    screen_width = 864
    screen_height = 936
    ground_scroll = 0
    scroll_speed = 4

    
    score = 0

    distance_between_pipes = 350
    pipe_gape = 200

    flying = True
    game_over = False
    pass_pipe = False

    pygame.init()
    font = pygame.font.SysFont('Bauhaus 93', 60)
    clock = pygame.time.Clock()
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption('Flappy Bird')

    background = pygame.image.load('img/bg.jpg')
    ground_img = pygame.image.load('img/ground_lunaire.png')

    first_tick = True


    ############################################################
    ################### BIRD ###################################
    ############################################################
    class Bird(pygame.sprite.Sprite):
        def __init__(self,x,y):
            pygame.sprite.Sprite.__init__(self)
            self.images = []
            self.index = 0
            self.counter = 0
            for num in range(1,4):
                img = pygame.image.load(f"img/bird{num}.png")
                self.images.append(img)
            self.image = self.images[self.index]
            self.rect = self.image.get_rect()
            self.rect.center = [x,y]
            self.vel = 0
            self.clicked = False

        def update(self, action=None):

            #gravity
            if flying == True:
                self.vel += 0.5
                if self.vel > 8:
                    self.vel = 8
                if self.rect.bottom < 768:
                    self.rect.y += int(self.vel)

            if game_over == False:

                #If we click to jump or if we use the AI to play the game
                ### JUMP
                if (pygame.mouse.get_pressed()[0] == 1 and self.rect.bottom > 0 and self.clicked == False):
                    self.clicked = True
                    if self.vel > -8:
                        self.vel = -10

                if (action == 1 and self.rect.bottom > 0 and self.clicked == False):
                    self.clicked = True
                    if self.vel > -8:
                        self.vel = -10

                #If we release the click or if the AI stop to jump
                ### STOP JUMP
                if pygame.mouse.get_pressed()[0] == 0 and self.clicked == True :
                    self.clicked = False

                if (action == 0 and self.clicked == True):
                    self.clicked = False


                #animate the bird
                self.counter += 1
                flap_cooldown = 10

                if self.counter > flap_cooldown:
                    self.counter = 0
                    self.index += 1
                    if self.index >= len(self.images):
                        self.index = 0
                    self.image = self.images[self.index]


                #rotate the bird
                self.image = pygame.transform.rotate(self.images[self.index], self.vel * -2)
            else:
                self.image = pygame.transform.rotate(self.images[self.index], -90)



    bird_group = pygame.sprite.Group()
    flappy = Bird(100, int(screen_height/2))
    bird_group.add(flappy)



    ############################################################
    ################### PIPES ##################################
    ############################################################

    class Pipe(pygame.sprite.Sprite):
        def __init__(self, x, y, position):
            pygame.sprite.Sprite.__init__(self)
            self.image = pygame.image.load("img/pipe_gray.png")
            self.rect = self.image.get_rect()
            #position 1 is from the top, -1 is from the bottom
            
            if position == 1:
                self.image = pygame.transform.flip(self.image, False, True)
                self.rect.bottomleft = [x,y-int(pipe_gape/2)]
            if position == -1:
                
                self.rect.topleft = [x,y+int(pipe_gape/2)]

        def update(self):
            self.rect.x -= scroll_speed
            if self.rect.right < 0:
                self.kill()


    pipe_group = pygame.sprite.Group()
    pipe_down = Pipe(screen_width, int(screen_height/2), 1)
    pipe_group.add(pipe_down)
    pipe_up = Pipe(screen_width, int(screen_height/2), -1)
    pipe_group.add(pipe_up)
    
    
    ############################################################
    ################### GAME ###################################
    ############################################################

    running = True
    while running:
        x_bird = flappy.rect.center[0]
        y_bird = flappy.rect.center[1]
        #print((x_bird, y_bird), flappy.rect.bottom, flappy.rect.top, flappy.rect.right, flappy.rect.left)
        
        
        
        clock.tick(FPS)
        screen.blit(background, (0, 0))
        screen.blit(ground_img, (ground_scroll, 768))

        bird_group.draw(screen)







        if first_tick:
            action = 1
        bird_group.update(action=action)
        pipe_group.draw(screen)

        #If the game is not over
        if game_over == False:

            ground_scroll -= scroll_speed
            if ground_scroll < -35:
                ground_scroll = 0

            #generate new pipes
        
            x_last_pipe = pipe_group.sprites()[-1].rect.center[0]
            #print(x_last_pipe)
            if screen_width - x_last_pipe > distance_between_pipes:

                pipe_height = random.randint(-200, 200)

                pipe_down = Pipe(screen_width, int(screen_height/2) + pipe_height, 1)
                
                pipe_group.add(pipe_down)
                pipe_up = Pipe(screen_width, int(screen_height/2) + pipe_height, -1)
                #print('new pipe', pipe_up.rect.center)
                pipe_group.add(pipe_up)
            pipe_group.update()
                
            #check if the bird hit the ground
            if flappy.rect.bottom > 768:
                game_over = True
                flying = False

            #check if the bird hits the roof
            if flappy.rect.top < 0:
                game_over = True
                

            #check if the bird hits a pipe
            if pygame.sprite.groupcollide(bird_group, pipe_group, False, False):
                game_over = True

            #check if the bird has gone through the pipe
            x_bird = flappy.rect.center[0]
            x_pipe_left = pipe_group.sprites()[0].rect.left
            x_pipe_right = pipe_group.sprites()[1].rect.right

            if x_pipe_left < x_bird < x_pipe_right and pass_pipe == False:
                pass_pipe = True

            if pass_pipe == True and x_bird > x_pipe_right:
                pass_pipe = False
                score += 1
                if verbose:
                    print(score)
            

        #Ask RL for the next action
        y_bird = flappy.rect.center[1]
        x_bird = flappy.rect.center[0]
        
        
        x_pipe0_right = pipe_group.sprites()[0].rect.right
        if x_pipe0_right < 100 - 30: #bird has passed the pipe
            next_pipe_height = pipe_group.sprites()[2].rect.center[1]
            next_pipe_x = pipe_group.sprites()[2].rect.center[0]

            # goes from 268 to 668
        
        else:
            next_pipe_height = pipe_group.sprites()[0].rect.center[1]
            next_pipe_x = pipe_group.sprites()[0].rect.center[0]
            # goes from 268 to 668


        
        delta_y = (next_pipe_height - y_bird, next_pipe_x-x_bird) #-1000 to 300

        print('ybird : ', y_bird, 'next_pipe_height', next_pipe_height, 'delta_y', delta_y)

        state = delta_y

        if first_tick:
            first_tick = False
            previous_state = delta_y
            previous_action = 1

        else:

            reward = 1
            if game_over:
                reward = -100


            previous_action = action

            Update_RL(previous_state, previous_action, reward, state)

            action = Action_RL(state)
            previous_state = state

        print(state)
                

            
        #If the game is over
        if game_over == True:
            running = False
              
        #display the score
        score_display = font.render(f'{score}', True, (255,255,255))
        screen.blit(score_display, (int(screen_width/2) ,10))

        

        #Leave the game
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        #print(bird_group.sprites()[0].rect.center, bird_group.sprites()[0].rect.bottom, bird_group.sprites()[0].rect.top, bird_group.sprites()[0].rect.right, bird_group.sprites()[0].rect.left)
        #print(pipe_group.sprites()[0].rect.center, pipe_group.sprites()[0].rect.bottom, pipe_group.sprites()[0].rect.top, pipe_group.sprites()[0].rect.right, pipe_group.sprites()[0].rect.left)
        #print(pipe_group.sprites()[1].rect.center, pipe_group.sprites()[1].rect.bottom, pipe_group.sprites()[1].rect.top, pipe_group.sprites()[1].rect.right, pipe_group.sprites()[1].rect.left)
        
        pygame.display.update()
    
    pygame.quit()
    return score

launch_game(75)


ybird :  468 next_pipe_height 88 delta_y (-380, 799)
(-380, 799)
ybird :  459 next_pipe_height 88 delta_y (-371, 795)
(-371, 795)
ybird :  450 next_pipe_height 88 delta_y (-362, 791)
(-362, 791)
ybird :  442 next_pipe_height 88 delta_y (-354, 787)
(-354, 787)
ybird :  434 next_pipe_height 88 delta_y (-346, 783)
(-346, 783)
ybird :  427 next_pipe_height 88 delta_y (-339, 779)
(-339, 779)
ybird :  420 next_pipe_height 88 delta_y (-332, 775)
(-332, 775)
ybird :  414 next_pipe_height 88 delta_y (-326, 771)
(-326, 771)
ybird :  408 next_pipe_height 88 delta_y (-320, 767)
(-320, 767)
ybird :  403 next_pipe_height 88 delta_y (-315, 763)
(-315, 763)
ybird :  398 next_pipe_height 88 delta_y (-310, 759)
(-310, 759)
ybird :  394 next_pipe_height 88 delta_y (-306, 755)
(-306, 755)
ybird :  390 next_pipe_height 88 delta_y (-302, 751)
(-302, 751)
ybird :  387 next_pipe_height 88 delta_y (-299, 747)
(-299, 747)
ybird :  384 next_pipe_height 88 delta_y (-296, 743)
(-296, 743)
ybird :  382 next_pipe_he

6

In [14]:
pygame.quit()

# Accelerate the game 

In [13]:
num_episodes = 1
for i in range(num_episodes):
    score = launch_game(500, verbose=False)
    print('n°', i, 'score :', score)

ybird :  468 next_pipe_height 88 delta_y (-380, 799)
(-380, 799)
ybird :  459 next_pipe_height 88 delta_y (-371, 795)
(-371, 795)
ybird :  450 next_pipe_height 88 delta_y (-362, 791)
(-362, 791)
ybird :  442 next_pipe_height 88 delta_y (-354, 787)
(-354, 787)
ybird :  434 next_pipe_height 88 delta_y (-346, 783)
(-346, 783)
ybird :  427 next_pipe_height 88 delta_y (-339, 779)
(-339, 779)
ybird :  420 next_pipe_height 88 delta_y (-332, 775)
(-332, 775)
ybird :  414 next_pipe_height 88 delta_y (-326, 771)
(-326, 771)
ybird :  408 next_pipe_height 88 delta_y (-320, 767)
(-320, 767)
ybird :  403 next_pipe_height 88 delta_y (-315, 763)
(-315, 763)
ybird :  398 next_pipe_height 88 delta_y (-310, 759)
(-310, 759)
ybird :  394 next_pipe_height 88 delta_y (-306, 755)
(-306, 755)
ybird :  390 next_pipe_height 88 delta_y (-302, 751)
(-302, 751)
ybird :  387 next_pipe_height 88 delta_y (-299, 747)
(-299, 747)
ybird :  384 next_pipe_height 88 delta_y (-296, 743)
(-296, 743)
ybird :  382 next_pipe_he